# Week 3 Mini Project 2: Product Demand Forecasts

In [1]:
# Just to ignore warnings

import warnings
warnings.filterwarnings('ignore')

# 1. Download the Dataset

Download the Dataset from the following link: \
https://www.kaggle.com/felixzhao/productdemandforecasting


# 2. Read the Dataset

Read the dataset into a Pandas Dataframe.\

In [2]:
import pandas as pd

df = pd.read_csv('Historical Product Demand.csv')

initial_length = len(df)

print('Initial dataset length: ',initial_length)


df

Initial dataset length:  1048575


,Product_Code,Warehouse,Product_Category,Date,Order_Demand
0,Product_0993,Whse_J,Category_028,2012/7/27,100
1,Product_0979,Whse_J,Category_028,2012/1/19,500
2,Product_0979,Whse_J,Category_028,2012/2/3,500
3,Product_0979,Whse_J,Category_028,2012/2/9,500
4,Product_0979,Whse_J,Category_028,2012/3/2,500
...,...,...,...,...,...
1048570,Product_1791,Whse_J,Category_006,2016/4/27,1000
1048571,Product_1974,Whse_J,Category_006,2016/4/27,1
1048572,Product_1787,Whse_J,Category_006,2016/4/28,2500
1048573,Product_0901,Whse_J,Category_023,2016/10/7,50


Does the dataset include any missing values? If so, drop them. \
Hint: Pandas can do that with one line of code!

In [3]:
df.isna().sum(axis = 0)

Product_Code            0
Warehouse               0
Product_Category        0
Date                11239
Order_Demand            0
dtype: int64

In [4]:
# Drop the missing values in the date column

df.dropna(subset=['Date'], inplace = True)

adjusted_date_length = len(df)

print('Total rows with missing dates = ', initial_length - adjusted_date_length)

Total rows with missing dates =  11239


In [5]:
# sort the data according to date column
df.sort_values('Date', ignore_index=True, inplace=True)
df.head()

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
0,Product_0965,Whse_A,Category_006,2011/1/8,2
1,Product_0412,Whse_S,Category_007,2011/10/20,(2)
2,Product_0125,Whse_S,Category_011,2011/10/20,(2)
3,Product_0642,Whse_C,Category_019,2011/10/31,3
4,Product_2137,Whse_S,Category_009,2011/11/18,(25)


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1037336 entries, 0 to 1037335
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   Product_Code      1037336 non-null  object
 1   Warehouse         1037336 non-null  object
 2   Product_Category  1037336 non-null  object
 3   Date              1037336 non-null  object
 4   Order_Demand      1037336 non-null  object
dtypes: object(5)
memory usage: 39.6+ MB


In [7]:
# Clearly there are negative numbers in Order_Demand column. Two ways to solve this problem:
# 1- Remove those rows and check. If they are small number compared to total rows then we are good.
# 2- Assume  an order amount entered as negative - within parenthesis by mistake. Thus just replace the parenthesis

# Also, Order_Demand has data type 'Object'

In [8]:
# 1- Remove rows with parenthesis

df.drop(df[df['Order_Demand'].str.startswith("(")].index, inplace = True)

# Convert the datatype of Order_Demand from Object to Integer
df.Order_Demand = df.Order_Demand.astype('int64')

OD_adjusted_length = len(df)

print('Total rows with parenthesis Order_Demand = ', adjusted_date_length-OD_adjusted_length)

Total rows with parenthesis Order_Demand =  5899


In [9]:
# Only 5899 rows out of over a million row dataset. Thus I choose to remove the rows with parenthisis

# 2- Remove parenthesis and set Order_Demand data type to Int

#df['Order_Demand']=df['Order_Demand'].str.replace('(',"").str.replace(')',"").astype(int)

#df

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1031437 entries, 0 to 1037335
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   Product_Code      1031437 non-null  object
 1   Warehouse         1031437 non-null  object
 2   Product_Category  1031437 non-null  object
 3   Date              1031437 non-null  object
 4   Order_Demand      1031437 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 47.2+ MB


In [11]:
df.Order_Demand.describe()

count    1.031437e+06
mean     4.962992e+03
std      2.911306e+04
min      0.000000e+00
25%      2.000000e+01
50%      3.000000e+02
75%      2.000000e+03
max      4.000000e+06
Name: Order_Demand, dtype: float64

# 3. Extract Features

Exclude the region and date from the considered features.\
Hint: You can choose to use all the features.


In [12]:
df.drop(['Date', 'Warehouse'], axis = 1, inplace = True)
df

,Product_Code,Product_Category,Order_Demand
0,Product_0965,Category_006,2
3,Product_0642,Category_019,3
9,Product_0980,Category_028,4000
10,Product_0965,Category_006,1
11,Product_0965,Category_006,3
...,...,...,...
1037331,Product_0689,Category_011,200
1037332,Product_1970,Category_005,2000
1037333,Product_1904,Category_019,4000
1037334,Product_0471,Category_015,30


# 4. Perform Preprocessing

Perform any needed pre-processing on the chosen features including: \
• Scaling. \
• Encoding. \
Dealing with Nan values. \

Hint:\
Use only the preprocessing steps you think are useful.

In [13]:
from sklearn.preprocessing import LabelEncoder
df.Order_Demand = LabelEncoder().fit_transform(df.Order_Demand)

In [14]:
# We can use Ordinal Encoder to encode the Product_Code and Product_Category

from sklearn.preprocessing import OrdinalEncoder
encoder=OrdinalEncoder()
df[['Product_Code', 'Product_Category']] = encoder.fit_transform(df[['Product_Code', 'Product_Category']])
df

,Product_Code,Product_Category,Order_Demand
0,954.0,5.0,2
3,641.0,18.0,3
9,969.0,27.0,1906
10,954.0,5.0,1
11,954.0,5.0,3
...,...,...,...
1037331,688.0,10.0,200
1037332,1958.0,4.0,1467
1037333,1892.0,18.0,1906
1037334,470.0,14.0,30


In [15]:
# assign attributues to X and y variables
y = df.Order_Demand.values

features = len(df.columns)
X = df.iloc[:, 1:features].values
X

array([[   5.,    2.],
       [  18.,    3.],
       [  27., 1906.],
       ...,
       [  18., 1906.],
       [  14.,   30.],
       [   6.,  148.]])

In [16]:
from sklearn.preprocessing import StandardScaler

X_scaled = StandardScaler().fit_transform(X)

# 5. Split the Data

Split your data as follows: \
• 80% training set \
• 10% validation set \
• 10% test set

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2, random_state = 0)
print(len(X_train), len(X_test))

825149 206288


In [18]:
X_validate, X_test, y_validate, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=0)
print(len(X_train), len(X_test), len(X_validate))

825149 103144 103144


# Training K-Nearest Neighbor (KNN) Regression

Use a KNN regressor model to train your data.


In [19]:
from sklearn.neighbors import KNeighborsRegressor
# Choose value of K
K=2
model=KNeighborsRegressor(n_neighbors=K)
model.fit(X_train, y_train)
    
score = model.score(X_test,y_test)
print(score)

0.9999991449288744


Choose the best k for the KNN algorithm by trying different values and validating performance on the validation set.

In [20]:
scores = []
score = 0
best_score = 0

K = range(1,4)

for i in K:
    modelKNN = KNeighborsRegressor(n_neighbors = i).fit(X_train, y_train)
    
    score = modelKNN.score(X_test, y_test)
    score = score
    scores.append(score)
    
    # Optionally we can run the below code instead
    if score > best_score :
        best_score = score
        bestK = i
        best_model = modelKNN # key advantage to the below code is saving the model
    
    print(i, "gives a score of:", score)
    
best_score = max(scores)
bestK = scores.index(best_score) + 1

print("KNN Regression gives a score of:", best_score, "with", bestK, "neighbors.") 

1 gives a score of: 0.9999993982600149
2 gives a score of: 0.9999991449288744
3 gives a score of: 0.9999992539315983
KNN Regression gives a score of: 0.9999993982600149 with 1 neighbors.


In [22]:
# Using the best model n_neighbors = 4
# best_model = KNeighborsRegressor(n_neighbors = 4).fit(X_train, y_train)

# Alternatively using the best_model acquired from the if statement
accuracy = best_model.score(X_validate, y_validate)
print('Accuracy of the best model = ', accuracy)

Accuracy of the best model =  0.9999975888471765


# Regression Metrics
Print the R-squared score of your final KNN regressor.

In [23]:
y_pred = best_model.predict(X_test)

from sklearn.metrics import r2_score
print('R-squared score of the final KNN regressor = ' + str(r2_score(y_test,y_pred)))

R-squared score of the final KNN regressor = 0.9999993982600149


# 7. Challenge Yourself (Optional)
Repeat step 6 for a different regression modelling technique.


In [24]:
#Using Linear Regression

from sklearn.linear_model import LinearRegression
modelLR=LinearRegression()
modelLR.fit(X_train, y_train)
    
y_pred=modelLR.predict(X_test)
    
score=modelLR.score(X_test,y_test)

print('R-squared score of the Linear regressor = ' + str(r2_score(y_test,y_pred)))
print('Accuracy with Linear regression = ', score)

R-squared score of the Linear regressor = 1.0
Accuracy with Linear regression =  1.0
